In [ ]:
# pip install tensorflow_datasets
# pip install magenta
# pip install music21
# pip install basic-pitch
# pip install aubio

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
from music21 import converter, corpus, instrument, midi, note, chord, pitch

from music21 import percussion
from music21.midi.percussion import MIDIPercussionException, PercussionMapper

PERCUSSION_MAPPER = PercussionMapper()

def open_midi(midi_path):
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()       
    print(len(mf.tracks)) # -- 1
    return midi.translate.midiFileToStream(mf)

midi_data = open_midi("../../data/tmp_groove/1_funk-groove1_138_beat_4-4.mid")
# midi_data # -- <music21.stream.Score ...>
print(len(midi_data.flatten().notesAndRests))

In [ ]:
# def extract_notes(midi_part):
#     parent_element = []
#     ret = []
#     for nt in midi_part.flat.notes:        
#         if isinstance(nt, note.Note):
#             ret.append(max(0.0, nt.pitch.ps))
#             parent_element.append(nt)
#         elif isinstance(nt, chord.Chord):
#             for pitch in nt.pitches:
#                 ret.append(max(0.0, pitch.ps))
#                 parent_element.append(nt)
    
#     return ret, parent_element
from music21 import stream

temp_midi = stream.Score()
temp_midi.insert(0, temp_midi_chords)

notes = midi_data.measures(0, 1).parts.flat.notes
midi_data.measures(0, 1).show("text")
print(len(notes))
for nt in notes:
    # print(nt)
    print(isinstance(nt, chord.Chord))
    # print(nt.pitch)
# top = base_midi.measures(10, 80).parts.flat.notes.pitch              
# y, parent_element = extract_notes(top)

In [ ]:
if isinstance(nr, note.Unpitched):
    try:
        i = PERCUSSION_MAPPER.midiPitchToInstrument(eOn.pitch)
    except MIDIPercussionException:
        # warnings.warn(str(mpe), TranslateWarning)
        i = instrument.UnpitchedPercussion()
    nr.storedInstrument = i
    # TODO: set reasonable displayPitch?

In [ ]:
def list_instruments(midi):
    partStream = midi.parts.stream()
    print("List of instruments found on MIDI file:")
    for p in partStream:
        aux = p
        print (p.partName)

list_instruments(base_midi)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

def extract_notes(midi_part):
    parent_element = []
    ret = []
    for nt in midi_part.flat.notes:        
        if isinstance(nt, note.Note):
            ret.append(max(0.0, nt.pitch.ps))
            parent_element.append(nt)
        elif isinstance(nt, chord.Chord):
            for pitch in nt.pitches:
                ret.append(max(0.0, pitch.ps))
                parent_element.append(nt)
    
    return ret, parent_element

def print_parts_countour(midi):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 1, 1)
    minPitch = pitch.Pitch('C10').ps
    maxPitch = 0
    xMax = 0
    
    # Drawing notes.
    for i in range(len(midi.parts)):
        top = midi.parts[i].flat.notes                  
        y, parent_element = extract_notes(top)
        if (len(y) < 1): continue
            
        x = [n.offset for n in parent_element]
        ax.scatter(x, y, alpha=0.6, s=7)
        
        aux = min(y)
        if (aux < minPitch): minPitch = aux
            
        aux = max(y)
        if (aux > maxPitch): maxPitch = aux
            
        aux = max(x)
        if (aux > xMax): xMax = aux
    
    for i in range(1, 10):
        linePitch = pitch.Pitch('C{0}'.format(i)).ps
        if (linePitch > minPitch and linePitch < maxPitch):
            ax.add_line(mlines.Line2D([0, xMax], [linePitch, linePitch], color='red', alpha=0.1))            

    plt.ylabel("Note index (each octave has 12 notes)")
    plt.xlabel("Number of quarter notes (beats)")
    plt.title('Voices motion approximation, each color is a different instrument, red lines show each octave')
    plt.show()

# Focusing only on 6 first measures to make it easier to understand.
print_parts_countour(base_midi.measures(0, 6))

In [ ]:
# # tfds works in both Eager and Graph modes
# # tf.enable_eager_execution()
# # print(tf.executing_eagerly()) # -- True

# # Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
# dataset = tfds.load(
#     name="groove/full-midionly",
#     split=tfds.Split.TRAIN,
#     try_gcs=True)

# """
# Split	        Examples
# 'test'	        129
# 'train'	        897
# 'validation'	124

# FeaturesDict({
#     'bpm': int32,
#     'drummer': ClassLabel(shape=(), dtype=int64, num_classes=10),
#     'id': string,
#     'midi': string,
#     'style': FeaturesDict({
#         'primary': ClassLabel(shape=(), dtype=int64, num_classes=18),
#         'secondary': string,
#     }),
#     'time_signature': ClassLabel(shape=(), dtype=int64, num_classes=5),
#     'type': ClassLabel(shape=(), dtype=int64, num_classes=2),
# })
# """

In [ ]:
# # print(dataset)
# # # Build your input pipeline
# # dataset = dataset.shuffle(1024).batch(32).prefetch(
# #     tf.data.experimental.AUTOTUNE)
# for features in dataset.take(1):
#   # Access the features you are interested in
#   midi, genre = features["midi"], features["style"]["primary"]
#   print(midi)